In [ ]:
import os
import pickle
from tqdm import tqdm
import numpy as np
import torch
from torch.autograd import Variable
import torch.nn as nn
from torch.nn.parameter import Parameter

from allensdk.core.brain_observatory_cache import BrainObservatoryCache
import allensdk.brain_observatory.receptive_field_analysis.visualization as rfvis
import allensdk.brain_observatory.receptive_field_analysis.receptive_field as rf
from allensdk.brain_observatory.receptive_field_analysis.eventdetection import detect_events
from sklearn import linear_model
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
from allensdk.core.brain_observatory_cache import BrainObservatoryCache
boc = BrainObservatoryCache(manifest_file='boc/manifest.json')

In [ ]:
# Specify output directories and cache results
def cached(results_dir, results_name):
    def _cache(func):
        def func_wrapper(*args, **kwargs):
            results_file = os.path.join(results_dir, results_name)
            if not results_file.endswith(".pkl"):
                results_file += ".pkl"

            if os.path.exists(results_file):
                with open(results_file, "rb") as f:
                    results = pickle.load(f)
            else:
                results = func(*args, **kwargs)
                with open(results_file, "wb") as f:
                    pickle.dump(results, f)

            return results
        return func_wrapper

    return _cache

In [ ]:
# ids = [501474098,
#  501847516]

ids = [
    501474098,
    501847516,
    505693621,
    501773889,
    511977695,
    530047022,
    503864409,
    502383036,
    540168837,
    527583578,
    528693630,
    539540432,
    510535700,
    510656082,
    511305590,
    502665019,
    530739576,
    505811062,
    501879034,
    502483554,
    506353473,
    508220632,
    537153918
]

num_experiments = len(ids)

#note: 23 total, since each of the 6 cre-lines only has 1-3 different depths and 1-4 different brain locations


In [ ]:
def get_data(i):
    data = boc.get_ophys_experiment_data(i)
    ts, dff = data.get_dff_traces()
    T = ts.shape[0]
    stimulus_table = data.get_stimulus_table("locally_sparse_noise")
    stimulus_template = data.get_stimulus_template("locally_sparse_noise")[stimulus_table['frame'].values, :, :]
    T_frames, H_px, W_px = stimulus_template.shape
    stim_reshaped = stimulus_template.reshape((T_frames, -1))
    x = np.zeros((T, H_px * W_px))
    for t in range(T_frames):
        x[stimulus_table.start[t]:stimulus_table.end[t]] = stim_reshaped[t]
    y = dff.T
    x -= x.mean()
    x /= x.std()
    y = dff.T
    y -= y.mean(axis=0)
    y /= y.std(axis=0)
    return x, y

In [ ]:
X, Y, num_neurons = [], [], []
for i in ids:
    f = cached("data", "{}.pkl".format(i))(get_data)
    xi, yi = f(i)
    X.append(xi)
    Y.append(yi)
    num_neurons.append(yi.shape[1])

In [ ]:
# split into test and train
X_train = []
X_test = []
y_train = []
y_test = []
for i in range(len(X)):
    X_tr, X_te, y_tr, y_te = train_test_split(X[i][:-10], Y[i][10:], test_size=0.80, random_state=42)
    X_train.append(X_tr)
    X_test.append(X_te)
    y_train.append(y_tr)
    y_test.append(y_te)

In [ ]:
from sklearn import linear_model

def linear_(X, Y):
    lr = LinearRegression()
    lr.fit(X, Y)
    return lr

def linear_with_neurons(X, Y, i):

    neurons = np.column_stack((Y[:,:i], Y[:,i+1:]))
    X_with_neurons = np.column_stack((X, neurons))
        
    lr = LinearRegression()
    lr.fit(X_with_neurons, Y[:,i])

    return lr

In [ ]:
# Fit linear regressions to each dataset
all_lrs = []
for j, idj in enumerate(ids):
    fit = cached("results", "linear_{}_{}.pkl".format(idj, j))(linear_)
    lr = fit(X_train[j], y_train[j])
    print("Dataset ", idj)
    all_lrs.append(lr)

In [ ]:
# # Compute train losses for each dataset 
# train_scores = []
# train_lps = []
# for xtr, ytr, lrs in zip(X_train, y_train, all_lrs):
#     T, N = ytr.shape
#     score = 0
#     lp = 0
#     for i in range(N):
#         score += (lrs[i].score(xtr, ytr[:,i]))
#         lp += lrs[i].predict_log_proba(xtr)[np.arange(T), ytr[:,i].astype(int)].sum()
#     train_scores.append(score / N)
#     train_lps.append(lp / (T*N))

    
# # Compute test losses for each dataset
# test_scores = []
# test_lps = []
# for xte, yte, lrs in zip(X_test, y_test, all_lrs):
#     T, N = yte.shape
#     score = 0
#     lp = 0
#     for i in range(N):
#         score += (lrs[i].score(xte, yte[:,i]))
#         lp += lrs[i].predict_log_proba(xte)[np.arange(T), yte[:,i].astype(int)].sum()
#     test_scores.append(score / N)
#     test_lps.append(lp / (T*N))

In [ ]:
# Fit logistic regressions to each dataset
all_lrs2 = []
for j, idj in enumerate(ids):
    Nj = Y[j].shape[1]
    lrs_j = []
    for i in range(Nj):
        fit = cached("results", "linearwithneurons_{}_{}.pkl".format(idj, i))(linear_with_neurons)
        lr = fit(X_train[j], y_train[j], i)
        lrs_j.append(lr)
        print("Dataset ", j, " neuron ", i)
    all_lrs2.append(lrs_j)

In [ ]:
# # Compute train losses for each dataset 
# train_scores2 = []
# train_lps2 = []
# for xtr, ytr, lrs in zip(X_train, y_train, all_lrs2):
#     T, N = ytr.shape
#     score = 0
#     lp = 0
#     for i in range(N):
#         xtr_i = np.column_stack((xtr, ytr[:,:i], ytr[:,i+1:]))
#         score += (lrs[i].score(xtr_i, ytr[:,i]))
#         lp += lrs[i].predict_log_proba(xtr_i)[np.arange(T), ytr[:,i].astype(int)].sum()
#     train_scores2.append(score / N)
#     train_lps2.append(lp / (T*N))

    
# # Compute test losses for each dataset
# test_scores2 = []
# test_lps2 = []
# for xte, yte, lrs in zip(X_test, y_test, all_lrs2):
#     T, N = yte.shape
#     score = 0
#     lp = 0
#     for i in range(N):
#         xte_i = np.column_stack((xte, yte[:,:i], yte[:,i+1:]))
#         score += (lrs[i].score(xte_i, yte[:,i]))
#         lp += lrs[i].predict_log_proba(xte_i)[np.arange(T), yte[:,i].astype(int)].sum()
#     test_scores2.append(score / N)
#     test_lps2.append(lp / (T*N))

In [ ]:
def get_corr_matrices(lrs):
    corr = []

    for lr in lrs:
        corr.append(lr.coef_[448:])
    corr = np.asarray(corr)
    return corr

In [ ]:
for i,lrs in enumerate(all_lrs2):
    lr = cached("correlations", "corr_matrix_{}_{}.pkl".format(ids[i], i))(get_corr_matrices)
    plt.figure()
    plt.imshow(lr, cmap="RdBu", vmin=.4)